In [2]:
import Main
import importlib
import copy
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize
import wcopt
import time
import estimation as est
import multiprocess as mp
import warnings
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            

importlib.reload(Main)
importlib.reload(wcopt)

from IPython.core.display import display, HTML
#display(HTML("<style>.container { width:80% !important; }</style>"))

# Initialization

In [ ]:
# Parameters
T          = 300  # Truncation horizon
runcode    = 1
inputs, unknowns, targets, outputs, theta0, shock_param, block_list = Main.getruncodeFeatures(runcode)

In [ ]:
# Run the model to initialize
ss, G, inputs, outputs_all, block_list = Main.SolveModel(runcode, theta0, T, [], inputs, outputs, unknowns, targets, block_list, False)

In [ ]:
G.keys()

In [ ]:
print(G['C']['rstar'])
print(G['Y']['markup'])

# Output information about the initialization
print(outputs)
print(outputs_all)

# Testing

In [ ]:
## Testing
importlib.reload(Main)

Npd  = 100
Nsim = 500

inputs, outputs, theta0 = Main.getruncodeFeatures(runcode)
outputs_all = inputs + outputs
mZs         = Main.getShockMACoeffs(runcode, T)


# Compare simulation from MA rep vs. simulation via direct draw
print("From MA")
dY  = Main.SimModel(ss, G, Npd, T, Nsim, inputs, outputs_all, mZs, True, None)  # From MA
print("From Drawing")
dY_ = Main.SimModel(ss, G, Npd, T, Nsim, inputs, outputs_all, mZs, False, None) # From draw
plt.plot(dY[:,:,1])
plt.show()
plt.plot(dY_[:,:,1])
plt.show()


# Compute moments given simulation
print("Moments")
muhats  = Main.compute_muhat(runcode, dY[:, len(inputs):, :])
print(np.mean(muhats, axis=1))
muhats_ = Main.compute_muhat(runcode, dY_[:, len(inputs):, :])
print(np.mean(muhats_, axis=1))


mXs_byo_byi, mXs_byi, mXs = Main.getEndogMACoeffs(G, mZs, inputs, outputs)
Sigma  = est.all_covariances(mXs, np.ones(len(inputs)))

print(np.vstack([np.diag(Sigma[0,:,:]), np.diag(np.cov(dY[:,len(inputs):,1].T))]).T)



# # Test analytical h
# print('Testing analytical h')
# print(Main.h_analytical(G, inputs, outputs, mZs, runcode))


# # Test feeder h
# print("")
# print('Testing h, empirical, sim from MA')
# print(Main.h([], ss, Npd, T, inputs, outputs, runcode, empirical=True, simFromMA=True, Nsim=Nsim))

# print('Testing h, empirical, direct draw')
# print(Main.h([], ss, Npd, T, inputs, outputs, runcode, empirical=True, simFromMA=False, Nsim=Nsim))

# print('Testing analytical h')
# print(Main.h([], ss, Npd, T, inputs, outputs, runcode, empirical=False))

# Running

## Simulation

In [5]:
# Simulate and Save
importlib.reload(Main)
runcode = 1
T       = 300
Npd     = 300  # Number of periods to simulate
Nsim    = 1000
fromMA  = True

resim = True
if resim:
    Main.muhatsSimSave(runcode, T, Npd, Nsim, fromMA, save=True, verbose=False)

Recomputing steady state


In [ ]:
# Comparing simulations generated from MA vs. direct drawing
simCompare = True
T       = 300
Nsim    = 1000
Npd     = 100  # Number of periods to simulate
if simCompare:
    savepath_MA  = "Results/muhats_%s.npy" % Main.getStrSave(runcode, T, Npd, Nsim, True)
    savepath_sim = "Results/muhats_%s.npy" % Main.getStrSave(runcode, T, Npd, Nsim, False)

    muhats_MA  = np.load(savepath_MA)
    muhats_sim = np.load(savepath_sim)

    print(np.vstack([muhats_MA.mean(axis=1), muhats_sim.mean(axis=1)]).transpose())
    print(100*(muhats_MA.mean(axis=1)-muhats_sim.mean(axis=1)) / muhats_sim.mean(axis=1))
    
plt.plot(muhats_MA[7,:])
plt.show()
plt.plot(muhats_sim[7,:])
plt.show()

## Identification

In [ ]:
# Ensuring that changing parameters changes the jacobians
resolveSS = True
if resolveSS:
    ss, G, inputs, outputs_all = Main.SolveModel([], T, [], False, runcode)

    inputs, outputs, theta0    = Main.getruncodeFeatures(runcode)
    theta0[0] = 2
    theta0[1] = 1
    theta0[2] = 1
    sspost, Gpost, inputs, outputs_all = Main.SolveModel(theta0, T, ss, False, runcode)
    
print(ss['Y'], ss['K'])
print(sspost['Y'], ss['K'])
print(np.hstack([G['Y']['Z'][:,0:2], Gpost['Y']['Z'][:,0:2]]))

In [ ]:
# Identification plots
importlib.reload(Main)

T       = 300
Npd     = 300  # Number of periods to simulate
checkID = True
if checkID:
    # Get parameters to loop over
    thetas, theta0 = Main.paramsCheckID(runcode)
    Nparams = len(thetas)
    
    # Evaluate at initial parameters
    Nmoments = len(Main.h(runcode, theta0, T, ss, Npd, inputs, outputs, unknowns, targets, block_list, True, False, 1000, False))
    

    # Loop over parameters
    for p in range(Nparams):
        print("Param %d / %d..." % (p+1, Nparams))
        
        # Set everything to defaults to start
        theta = copy.deepcopy(theta0)
        print(theta0)
        
        # Create array to hold values of moments across different parameter vals
        h_ = np.zeros((Nmoments, len(thetas[p])))
                 
        # Loop over values in identification range        
        for i in range(len(thetas[p])):
            # Set that parameter
            theta[p] = thetas[p][i]
            print(theta)
            
            # Compute h
            h_[:,i] = Main.h(runcode, theta, T, ss, Npd, inputs, outputs, unknowns, targets, block_list, True, False, 1000, False)

        # Plot identification plots by moment
        for m in range(Nmoments):
            plt.subplot(np.round(np.ceil(Nmoments/4)), 4, m+1)
            plt.plot(h_[m,:])
        plt.show()
        
        print(h_)

## Moment Matching

In [7]:
importlib.reload(wcopt)
importlib.reload(Main)

# Load moments and do moment-matching
runcode = 1
T       = 300
Npd     = 300  # Number of periods to simulate
Nsim    = 1000
fromMA  = False

hempirical = False
hfromMA    = False
seed       = 314


rematch    = True
if rematch:
    Main.MomentMatchRuns(runcode, T, Npd, Nsim, fromMA, hempirical, hfromMA)



Recomputing steady state
	Param 1 / 3
Matching Simulation 1 / 1000
Estimating G_true...
[[-3.82841990e-04 -3.36259116e-03  3.13077026e-02]
 [-4.60588482e-03  2.28038728e-04  1.88235719e-04]
 [-6.43008165e-04  3.27721931e-04 -2.96783061e-04]
 [-4.08381680e-04 -3.88625936e-03  3.06909471e-02]
 [-4.26445717e-03  1.14150689e-04  2.99036374e-04]
 [-6.13819091e-05 -5.78343400e-05  1.07104365e-04]
 [-8.75131356e-04 -3.28415797e-03 -3.78024335e-03]
 [ 3.71255079e-04  7.78643400e-04 -2.04560649e-03]
 [ 7.97393985e-04 -7.39299492e-05  2.57965896e-04]
 [-3.76650442e-02 -4.99419968e-02  5.96403770e-04]
 [-3.22845852e-08 -2.28986017e-07  4.83447104e-06]
 [-7.71131441e-05 -3.33521221e-04  3.68551472e-04]]
DONE
Estimating theta_naive...[1.84294321 0.5801205  0.76769363]
[1.69995793 0.42249598 0.71560019]
[1.48872613 0.16020926 0.67767571]
[1.52434588 0.13071494 0.68083488]
[1.5149672  0.11608294 0.67872617]
[1.53488931 0.10364233 0.68174137]
[1.54118556 0.11111542 0.68716175]
[1.54205527 0.11049036 0

KeyboardInterrupt: 

In [ ]:
print("Number of processors: ", mp.cpu_count())
joe = lambda : print("bob")
joe()
pool.close()
np.size([]) == 0